# Demonstrating different compute modes with standalone H2 electrolyzer

This notebook shows how to use different compute modes (`normal`, `feedstock_size`, and `product_size`) using a simple system with only one complex converter, and two simple converters connected to it.
These simple converters are the new `feedstock_profile` and `product_profile` class, and simply function as dictated energy/mass flow profiles that can be set in a .csv rather than calculated by a model.

## Compute modes

For the converters involved in this example, three different `compute_mode` parameters have been defined, which determine how the `PerformanceModel.compute()` function will execute and which variables to create in the `PerformanceModel.setup()` function.

- `normal`: In this mode, converters function as they always have previously:
    - The size of the asset is fixed within the `compute()` function, and is read from the config.
    - The feedstock profiles are fixed and set as the `<feedstock>_in` inputs, typically from a connection to another upstream converter.
    - The product profiles are calculated and set as the `<product>_out` outputs, and may be passed to another downstream converter.
- `feedstock_size`: In this mode, the size of the asset is adjusted to be able to utilize all of the available feedstock:
    - The size of the asset is *calculated* within the `compute()` function, by how big it needs to be to utilize all of the `<feedstock>_in`.
    - The feedstock profiles are fixed and set as the `<feedstock>_in` inputs, typically from a connection to another upstream converter - same as `normal` mode.
    - The product profiles are calculated and set as the `<product>_out` outputs, and may be passed to another downstream converter - same as `normal` mode.
- `product_size`: In this mode, the size of the asset is adjusted to be able to meet the demand profile for its product:
    - The size of the asset is calculated within the `compute()` function, by how big it needs to be to provide all of the `<product>_demand` - a new input that is defined in `setup()` just for this mode.
    - The feedstock profiles are *calculated* and set as the `<feedstock>_consume` *outputs*, and may be passed to another upstream converter.
    - The product profiles are *fixed* and set as the `<product>_demand` *inputs*, typically from a connection to another downstream converter.

## Example plant setup

Here, there are three technologies in the the `tech_config.yaml`: And `electrolyzer`, a `feedstock_profile` (configured for electricity), and a `product_profile` (configured for hydrogen).
The feedstock/product profiles are very simple converters that are set up to either read a .csv file that defines the profile OR function as a flexible input/output source of the commodity.
In both cases, a simple buying or selling price is set for the commodity to feed into financial calculations.

### Example technology_interconnections within plant_config.yaml
```yaml
technology_interconnections: [
  ["feedstock_profile", "electrolyzer", "electricity", "cable"],
  ["electrolyzer", "product_profile", "hydrogen", "pipe"]
]
```
In `normal` mode or `feedstock_size` mode, the interconnection will behave as they previously have - connecting from `<source_tech>.outputs["<commodity>_out"]` to `<dest_tech>.inputs["<commodity>_in"]`
However, if the destination technology is in `product_size` mode, the interconnection will instead be made from `<dest_tech>.outputs["<commodity>_demand"]` to `<source_tech>.inputs["<commodity>_consume"]`

### Example tech_config.yaml (some parameters skipped for brevity)
```yaml
technologies:
  electrolyzer:
    performance_model:
      model: "eco_pem_electrolyzer_performance"
    cost_model:
      model: "singlitico_electrolyzer_cost"
    model_inputs:
      performance_parameters:
        compute_mode: "product_size"
        size_by: "hydrogen"
  feedstock_profile:
    performance_model:
      model: "feedstock_performance"
    cost_model:
      model: "feedstock_cost"
    model_inputs:
      shared_parameters:
        commodity: "electricity"
      performance_parameters:
        compute_mode: "product_size"
        profile: "elec_profile.csv"
        unit: "kW"
      cost_parameters:
        buy_price: 0.07
  product_profile:
    performance_model:
      model: "product_performance"
    cost_model:
      model: "product_cost"
    model_inputs:
      shared_parameters:
        commodity: "hydrogen"
      performance_parameters:
        compute_mode: "product_size"
        profile: "h2_profile.csv"
        unit: "kg/h"
      cost_parameters:
        sell_price: 1.50
        is_coproduct: False
```

## Running an example

### `normal` mode

The example `.yaml` files have been set up in `normal` mode.
In this case, an electricity profile will be read in by the `feedstock_profile`, and the electrolyzer will utilize whatever `electricity_in` it can at the size that has been set in the config.
The `product_profile` will simply act as an "end cap" for the `hydrogen_out` profile to be passed to.

In this case the electrolyzer has been sized to 640 MW (same as previous example), but the electricity profile going in only has a max of 600 MW.
Thus, the LCOH ($5.33/kg) is not quite optimal - the electrolyzer size should be reduced.

In [17]:
from h2integrate.core.h2integrate_model import H2IntegrateModel


# Create a H2Integrate model
model = H2IntegrateModel("14_standalone_h2_pem_normal.yaml")

# Run the model
model.run()

model.post_process()

25 Input(s) in 'model'

varname                                    val                  units     prom_name                                             
-----------------------------------------  -------------------  --------  ------------------------------------------------------
plant
  feedstock_profile
    feedstock_performance
      electricity_profile                  |39730239.80618451|  kW        feedstock_profile.electricity_profile                 
    feedstock_cost
      electricity_out                      |39730239.80618451|  kW        feedstock_profile.electricity_out                     
  feedstock_profile_to_electrolyzer_cable
    electricity_in                         |39730239.80618451|  kW        feedstock_profile_to_electrolyzer_cable.electricity_in
  electrolyzer
    eco_pem_electrolyzer_performance
      electrolyzer_size_mw                 [640.]               MW        electrolyzer.electrolyzer_size_mw                     
      electricity_in                 

### `feedstock_size` mode

In this case, the electrolyzer will be sized to match the electricity supply profile going in thru `elec_profile.csv`.
This reduces the electrolyzer size to 600 MW, optimizing the LCOH to $5.27/kg.
Note that this does not require used of the optimizer, saving valuable computation time in a multivariable optimization.

In [18]:
# Create a H2Integrate model
feed_model = H2IntegrateModel("14_standalone_h2_pem_feedstock_size.yaml")

# Run the model
feed_model.run()

feed_model.post_process()

25 Input(s) in 'model'

varname                                    val                   units     prom_name                                             
-----------------------------------------  --------------------  --------  ------------------------------------------------------
plant
  feedstock_profile
    feedstock_performance
      electricity_profile                  |39730239.80618451|   kW        feedstock_profile.electricity_profile                 
    feedstock_cost
      electricity_out                      |39730239.80618451|   kW        feedstock_profile.electricity_out                     
  feedstock_profile_to_electrolyzer_cable
    electricity_in                         |39730239.80618451|   kW        feedstock_profile_to_electrolyzer_cable.electricity_in
  electrolyzer
    eco_pem_electrolyzer_performance
      electrolyzer_size_mw                 [640.]                MW        electrolyzer.electrolyzer_size_mw                     
      electricity_in           

### `product_size` mode

In this case, the electrolyzer will be sized to match the hydrogen demand profile going in thru `h2_profile.csv`.
The hydrogen demand profile is the same hydrogen output profile that was previously calculated in `feedstock_size` mode, to test if there is any difference going forward and backward.
It reduces the electrolyzer size to 600 MW, optimizing the LCOH to $5.27/kg, but with a slight difference in the trailing digits.
This is because in `product_size` the electrolyzer performance is calculated iteratively.
It starts with a best guess of the electricity profile required to meet the hydrogen demand, then calculates the residual between the hydrogen demand profile and the hydrogen produced.
The residuals are used to correct the electricity profile iteratively until a minimum max residual is achieved (below a ratio of 1e-6 between the maximum hydrogen produced and the maximum residual).
This results in a small but insignificant difference in the LCOH calculated in `feedstock_size` and `product_size` methods.

In [19]:
# Create a H2Integrate model
prod_model = H2IntegrateModel("14_standalone_h2_pem_product_size.yaml")

# Run the model
prod_model.run()

prod_model.post_process()

25 Input(s) in 'model'

varname                                    val                   units     prom_name                                             
-----------------------------------------  --------------------  --------  ------------------------------------------------------
plant
  product_profile
    product_performance
      hydrogen_profile                     |787626.80178713|     kg/h      product_profile.hydrogen_profile                      
    product_cost
      hydrogen_consume                     |787626.80178713|     kg/h      product_profile.hydrogen_consume                      
  electrolyzer_to_product_profile_pipe
    hydrogen_in                            |218.78522272|        kg/s      electrolyzer_to_product_profile_pipe.hydrogen_in      
  electrolyzer
    eco_pem_electrolyzer_performance
      electrolyzer_size_mw                 [640.]                MW        electrolyzer.electrolyzer_size_mw                     
      hydrogen_demand                   